In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
from scipy import ndimage
import cv2
import os
from skimage.transform import resize

In [52]:
num_px=64

In [53]:
print(os.listdir(r"C:\Users\lenovo\Desktop\kutta-billi\training_set"))

['cats', 'cattest', 'dogs', 'dogtest', 'New Folder']


In [54]:
def load(folder):
    images = []
    for filename in os.listdir(folder):
        image = cv2.imread(os.path.join(folder,filename))
        if image is not None:
            image = image/255.
            #my_image = skimage.transform.resize(image, size=(num_px,num_px)).resize((1, num_px*num_px*3)).T
            my_image=resize(image, (64,64), anti_aliasing=True)
            images.append(my_image)
    return images

In [55]:
def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

In [56]:
# taking dog for train
train_X_dog_list=load(r"C:\Users\lenovo\Desktop\kutta-billi\training_set\dogtest")
train_X_dog=np.array(train_X_dog_list)

In [57]:
train_X_cat_list=load(r"C:\Users\lenovo\Desktop\kutta-billi\training_set\cattest")
train_X_cat=np.array(train_X_cat_list)

In [58]:
train_X_orig = np.concatenate((train_X_dog, train_X_cat))
train_Y_dog = np.ones((train_X_dog.shape[0],1))
train_Y_cat = np.zeros((train_X_cat.shape[0],1))
train_Y_orig = np.append(train_Y_dog,train_Y_cat)
train_X, train_Y = shuffle_in_unison(train_X_orig, train_Y_orig)
train_X = train_X.reshape(train_X.shape[0], -1).T

In [59]:
test_X_dog_list=load(r"C:\Users\lenovo\Desktop\kutta-billi\training_set\dogs")
test_X_dog=np.array(test_X_dog_list)

In [60]:
test_X_cat_list=load(r"C:\Users\lenovo\Desktop\kutta-billi\training_set\cats")
test_X_cat=np.array(test_X_cat_list)

In [61]:
test_X_orig = np.concatenate((test_X_dog, test_X_cat))
test_Y_dog = np.ones((test_X_dog.shape[0],1))
test_Y_cat = np.zeros((test_X_cat.shape[0],1))
test_Y_orig = np.append(test_Y_dog,test_Y_cat) 
test_X, test_Y = shuffle_in_unison(test_X_orig, test_Y_orig)
test_X = test_X.reshape(test_X.shape[0], -1).T

In [37]:
def initialize_with_zeros(dim):
    
    
    
    w = np.zeros((dim,1))
    b = 0
    
    assert(w.shape == (dim, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    
    return w, b

In [62]:
def propagate(w, b, X, Y):
    
    m = X.shape[1]
    
    v=np.dot(w.T,X) + b
    A = 1/(1+np.exp(-v))
    cost = -1*np.sum(Y*(np.log(A))  + (1-Y)*(np.log(1-A)))/m    
    
    
   
               
    db = np.sum(A - Y)/m 
    dw = np.dot(X,(A - Y).T)/m
   

    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

In [63]:
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    
    costs = []
    
    for i in range(num_iterations):
        
        
        
        grads, cost = propagate(w, b, X, Y)
       
        dw = grads["dw"]
        db = grads["db"]
        
       
        w = w - learning_rate * dw
        b = b - learning_rate * db
        
        if i % 100 == 0:
            costs.append(cost)
        
        
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [64]:
def predict(w, b, X):
    
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)
    
    o=np.dot(w.T,X) + b
    A = 1/(1+np.exp(-o))
    
    
    for i in range(A.shape[1]):
        
        
        if A[0,i] <= 0.5:
            Y_prediction[0,i] = 0
        else:
            Y_prediction[0,i] = 1
     
    
    assert(Y_prediction.shape == (1, m))
    
    return Y_prediction

In [65]:
def model(X_train, Y_train, X_test, Y_test, num_iterations = 200, learning_rate = 0.5, print_cost = False):

    
    
    w, b = initialize_with_zeros(X_train.shape[0])

   
    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)
    
    
    w = parameters["w"]
    b = parameters["b"]
    
    
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)

    
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [66]:

c = model(train_X, train_Y, test_X, test_Y, num_iterations = 500, learning_rate = 0.005, print_cost = True)

Cost after iteration 0: 0.693147
Cost after iteration 100: 0.248350
Cost after iteration 200: 0.232778
Cost after iteration 300: 0.220196
Cost after iteration 400: 0.209620
train accuracy: 92.96296296296296 %
test accuracy: 93.25692963752665 %
